<a href="https://colab.research.google.com/github/IshaanKetchup/ASLtoEnglish/blob/main/PyTorch_CNN_on_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

# Loading the Dataset


In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# Loading in relevant libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Defining relevant variables for the ML task
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 20
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Using the inbuilt CIFAR10 dataset from torchvision (this is how the tutorial made me do it, so I stuck to it)

In [ ]:
# Used to reformat images for modelling,
# and save variable all_transforms for later use
all_transforms = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.4914,0.4822,0.4465],
                          std = [0.2023, 0.1994, 0.2010])
     ]
)

# Create training dataset
train_dataset = torchvision.datasets.CIFAR10(
    root = './data',
    train=True,
    transform = all_transforms,
    download = True
)

test_dataset = torchvision.datasets.CIFAR10(
    root = './data',
    train=False,
    transform = all_transforms,
    download = True
)

# Loader objects to facilitate processing
train_loader = torch.utils.data.DataLoader(
    dataset = train_dataset,
    batch_size = batch_size,
    shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    dataset = test_dataset,
    batch_size = batch_size,
    shuffle = True
)

Files already downloaded and verified
Files already downloaded and verified


#Defining the CNN
*   Class that extends the nn.Module class (PyTorch). Needed as it provides useful methods.
*   Define layers in NN. Done in the __init__ method, where layers are named and assigned appropriate layer that we want
*   Forward method: to define order in which input data passes through layers







In [ ]:
class ConvNeuralNet(nn.Module):
  # Determine what layers and their order in CNN object
  def __init__(self, num_classes):
    super(ConvNeuralNet, self).__init__()
    self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
    self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
    self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
    self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
    self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.fc1 = nn.Linear(1600, 128)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(128, num_classes)

  def forward(self,x):
    out = self.conv_layer1(x)
    out = self.conv_layer2(out)
    out = self.max_pool1(out)

    out = self.conv_layer3(out)
    out = self.conv_layer4(out)
    out = self.max_pool2(out)

    out = out.reshape(out.size(0), -1)
    out = self.fc1(out)
    out = self.relu1(out)
    out = self.fc2(out)
    return out

#Setting up hyperparameters
Choose cross-entropy as our loss function, and SGD (Stochastic Gradient Descent) as our optimizer
Also define a total_step to make iteration through various batches easier

In [ ]:
model = ConvNeuralNet(num_classes)

# Set Loss function with criterion
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)

total_step = len(train_loader)



#Training the model


1.   Start by iterating through epochs and then batches
2.   Convert images and labels according to device configured (GPU/CPU)
3. Forward pass: make predictions using our model and calculate loss based on those predictions and actual labels
4. Backward pass: update our weights to improve model
5. Set Gradients to 0 before every update using `optimizer.zero_grad()`
6. Calculate new gradients using `loss.backward()`
7. Update weights using `optimizer.step()`







In [ ]:
for epoch in range(num_epochs):
  model = model.to(torch.device('cuda'))
  # Load in the data in batches using tran_loader
  for i, (images,labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 0.3151
Epoch [2/20], Loss: 0.1227
Epoch [3/20], Loss: 0.1259
Epoch [4/20], Loss: 0.7392
Epoch [5/20], Loss: 0.3301
Epoch [6/20], Loss: 0.5891
Epoch [7/20], Loss: 0.5990
Epoch [8/20], Loss: 0.5488
Epoch [9/20], Loss: 0.6056
Epoch [10/20], Loss: 0.7046
Epoch [11/20], Loss: 0.5006
Epoch [12/20], Loss: 0.3136
Epoch [13/20], Loss: 0.2654
Epoch [14/20], Loss: 0.3677
Epoch [15/20], Loss: 0.2788
Epoch [16/20], Loss: 0.3517
Epoch [17/20], Loss: 0.2839
Epoch [18/20], Loss: 0.2991
Epoch [19/20], Loss: 0.0596
Epoch [20/20], Loss: 0.1721


# Testing the Model
`torch.no_grad()` as there is no need to calculate any gradients. Then predict each batch using our model and calculate how many it predicts correctly


In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the {} train images: {} %'.format(50000, 100 * correct / total))




Accuracy of the network on the 50000 train images: 94.43 %
